In [2]:
import pandas as pd

In [3]:
npr = pd.read_csv("C:\\Users\\fabia\\Desktop\\modelos\\NewsF.csv",sep=";")

In [4]:
npr.head()

,Unnamed: 0,Type,News
0,0,Business,Ad sales boost Time Warner profit\n\nQuarterly...
1,1,Business,Dollar gains on Greenspan speech\n\nThe dollar...
2,2,Business,Yukos unit buyer faces loan claim\n\nThe owner...
3,3,Business,High fuel prices hit BA's profits\n\nBritish A...
4,4,Business,Pernod takeover talk lifts Domecq\n\nShares in...


# Preprocesamiento

In [5]:
#Vectorizacion
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=0.9,min_df=2,stop_words='english')
dtm = cv.fit_transform(npr['News'])

In [6]:
dtm

<2225x17473 sparse matrix of type '<class 'numpy.int64'>'
	with 310493 stored elements in Compressed Sparse Row format>

# ICA Transformacion

In [7]:
from sklearn.decomposition import FastICA
ica_model = FastICA(n_components=5,random_state=42)

In [8]:
ica_model.fit(dtm.toarray())

FastICA(algorithm='parallel', fun='logcosh', fun_args=None, max_iter=200,
    n_components=5, random_state=42, tol=0.0001, w_init=None, whiten=True)

# Busqueda de cuadricula
(Para hacer prueba con diferents valores en modelos y encontrar el mejor)

In [9]:
search_params = {'n_components':[5,10,15],'random_state':[42]}

In [10]:
ica = FastICA()

In [11]:
from sklearn.model_selection import GridSearchCV
model = GridSearchCV(ica,param_grid=search_params)

In [12]:
model.fit(dtm.toarray())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator FastICA(algorithm='parallel', fun='logcosh', fun_args=None, max_iter=200,
    n_components=None, random_state=None, tol=0.0001, w_init=None,
    whiten=True) does not.

# Tema dominante en el texto

In [13]:
#Nombre de columnas
topicnames = []
for i in range(int(ica_model.n_components)):
    name='Topic'+str(i)
    topicnames.append(name)

In [14]:
#nombre de indices
docnames = []
#for i in range(len(cv.get_feature_names())):
for i in range(2225):
    docnames.append('Doc'+str(i))

In [15]:
import numpy as np
ica_output = ica_model.transform(dtm.toarray())
#Creando un dataframe en pandas
df_document_topic=pd.DataFrame(np.round(ica_output,2),columns=topicnames,index=docnames)

In [16]:
# Obtener tema dominante para cada documento
dominant_topic=np.argmax(df_document_topic.values,axis=1)
df_document_topic['dominant_topic']=dominant_topic

In [17]:
#Estilando
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

In [18]:
#Aplicando estilo
df_document_topic=df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.02,-0,-0.01,-0,-0.02,0
Doc1,-0,0.01,-0,-0.01,-0,1
Doc2,0,0.01,-0.02,-0,-0.01,1
Doc3,0.02,0.01,-0.01,-0,-0.01,0
Doc4,-0.01,0,-0.01,-0.01,-0.01,1
Doc5,-0.01,0,-0.01,-0.01,-0.01,1
Doc6,0.01,0.01,-0.01,-0,-0.01,0
Doc7,0,0.01,-0.01,-0.01,0,1
Doc8,-0.01,0,-0.01,-0,-0.01,1
Doc9,-0.01,0.01,-0.01,-0.01,-0,1


# Palabras clave del tema

In [19]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(ica_model.components_)

# Assign Column and Index
df_topic_keywords.columns = cv.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,00,000,000m,000th,007,01,0100,011,0130,02,...,zola,zombie,zombies,zone,zones,zoom,zooms,zurich,zvonareva,zvyagintsev
Topic0,0.000007,0.000226,-0.000017,-3.745315e-06,-6.503201e-07,-3.462830e-07,9.444984e-07,-1.712480e-07,0.000007,-1.040615e-06,...,-7.323717e-07,-5.680909e-06,1.637895e-05,7.181637e-06,-2.522128e-06,-2.298413e-06,1.111208e-06,0.000001,7.849376e-07,-1.432049e-06
Topic1,0.000003,-0.000149,0.000029,-1.047979e-07,5.754012e-06,2.769609e-06,-4.995766e-07,2.192020e-07,0.000010,1.359505e-06,...,1.563419e-06,-1.193480e-05,-4.224849e-07,7.399439e-06,-1.264098e-06,-7.636361e-06,-1.269485e-05,0.000004,3.907779e-06,9.341949e-07
Topic2,0.000023,0.000040,0.000057,-2.050392e-06,5.954245e-06,1.798662e-06,-3.030434e-06,-1.199848e-06,0.000001,2.114405e-06,...,3.579372e-06,-8.491204e-06,3.741200e-06,5.172217e-07,-1.555841e-06,-2.483485e-07,-6.077388e-06,0.000011,6.397697e-06,-5.657898e-07
Topic3,0.000003,-0.000025,0.000017,3.748569e-07,5.351305e-06,1.025199e-06,3.283636e-06,-2.634101e-07,0.000010,5.796337e-07,...,-2.801270e-07,7.834322e-07,-1.316158e-06,1.332577e-06,-7.965541e-07,-3.287150e-06,-7.047844e-07,-0.000002,1.488791e-06,7.622479e-07
Topic4,-0.000001,-0.000068,-0.000007,7.858411e-08,-1.901887e-06,-1.049129e-06,-9.048532e-07,-3.160422e-07,-0.000003,-1.560315e-07,...,-6.570106e-07,1.306634e-06,-7.675402e-06,-3.536676e-06,2.960572e-07,-6.961146e-07,-2.148085e-06,-0.000005,-2.006911e-06,-8.926163e-08


# 15 palabras clave principales de cada tema

In [20]:
#Obtener los temas correspondientes y las palabras relacionadas con cada tema
for index,topic in enumerate(ica_model.components_):
    print(f'The top 15 words for topic # {index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

The top 15 words for topic # 0
['pay', 'consumers', 'increase', 'net', 'lord', 'years', 'law', 'minimum', 'wage', 'firms', 'best', 'government', 'uk', 'people', 'said']


The top 15 words for topic # 1
['lord', 'british', 'aviator', '25', 'director', 'years', 'actress', 'actor', 'win', 'wales', 'song', 'england', 'film', 'said', 'best']


The top 15 words for topic # 2
['like', 'playing', 'roddick', 'just', 'play', 'wales', 'minimum', 'wage', 'england', 'world', 'people', 'new', 'time', 'games', 'game']


The top 15 words for topic # 3
['don', 'said', 'award', 'think', 'won', 'british', 'film', 'robbie', 'awards', 'angels', '25', 'music', 'years', 'song', 'best']


The top 15 words for topic # 4
['told', 'tory', 'music', 'kilroy', 'howard', 'chancellor', 'prime', 'minister', 'government', 'brown', 'party', 'election', 'blair', 'labour', 'mr']




In [21]:
# Show top n keywords for each topic
def show_topics(cv=cv, ica_model=ica_model, n_words=20):
    keywords = np.array(cv.get_feature_names())
    topic_keywords = []
    for topic_weights in ica_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(cv=cv, ica_model=ica_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,said,people,uk,government,best,firms,wage,minimum,law,years,lord,net,increase,consumers,pay
Topic 1,best,said,film,england,song,wales,win,actor,actress,years,director,25,aviator,british,lord
Topic 2,game,games,time,new,people,world,england,wage,minimum,wales,play,just,roddick,playing,like
Topic 3,best,song,years,music,25,angels,awards,robbie,film,british,won,think,award,said,don
Topic 4,mr,labour,blair,election,party,brown,government,minister,prime,chancellor,howard,kilroy,music,tory,told


# SOM

In [126]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from sompy.sompy import SOMFactory
from sklearn.datasets import fetch_california_housing

In [128]:
data = fetch_california_housing()
descr = data.DESCR
names = fetch_california_housing().feature_names+["HouseValue"]

data = np.column_stack([data.data, data.target])
print(descr)
print( "FEATURES: ", ", ".join(names))

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [ ]:
name=['Tech','Economy','Sport','Entertainment','Business']
#msz = calculate_msz(data)
sm = SOMFactory().build(ica_model.components_, normalization = 'var', initialization='random', component_names=name)
sm.train(n_job=-1, verbose=False, train_rough_len=2, train_finetune_len=5)

topographic_error = sm.calculate_topographic_error()
quantization_error = np.mean(sm._bmu[1])
print ("Topographic error = %s; Quantization error = %s" % (topographic_error, quantization_error))

In [ ]:
from sompy.visualization.mapview import View2D
view2D  = View2D(10,10,"rand data",text_size=10)
view2D.show(sm, col_sz=4, which_dim="all", denormalize=True)

In [ ]:
from sompy.visualization.bmuhits import BmuHitsView

vhts  = BmuHitsView(10,10,"Hits Map",text_size=7)
vhts.show(sm, anotate=True, onlyzeros=False, labelsize=10, cmap="autumn", logaritmic=False)
plt.show()
#cmap="Greys"

In [ ]:
#Kmeans cluster
from sompy.visualization.hitmap import HitMapView
sm.cluster(5)
hits  = HitMapView(10,10,"Clustering",text_size=7)
a=hits.show(sm, labelsize=12)